# Importation des bibliothéques

In [ ]:
import networkx as nx
import random
import string
from node2vec import Node2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

# Classification

In [ ]:
G = None #Création d'un graph avec networkx

# Générer les embeddings de noeuds avec Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Obtenir les embeddings de noeuds appris
node_embeddings = model.wv

labeled_nodes = None # La liste des noeuds qui ont des labels
one_hot_labels = None # Convertir les étiquettes en vecteurs One-Hot Encoding
total_label_count = None # Nombre total de labels (pfam)

# Créer les caractéristiques X et les étiquettes y pour la classification
X = np.array([node_embeddings.get_vector(str(node)) for node in labeled_nodes])
y = np.array([one_hot_labels.get(node, [0] * total_label_count) for node in labeled_nodes])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création du classificateur MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100, activation='relu', solver='adam', random_state=42)

classifier.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred = classifier.predict(X_test)

# Calculer l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Grid search 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définition des hyperparamètres à rechercher
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'max_iter': [100, 200]
}

# Création de l'objet GridSearchCV
grid_search = GridSearchCV(MLPClassifier(random_state=42), param_grid, cv=5)

# Recherche des meilleurs hyperparamètres
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

# Obtention du meilleur modèle
best_model = grid_search.best_estimator_

# Prédire les étiquettes pour les données de test avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Calculer l'exactitude avec le meilleur modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy avec le meilleur modèle:", accuracy)
